In [1]:
import time
import collections
import random
import numpy as np
import gym
import keras.layers
import keras.models
from keras import backend as K

Using TensorFlow backend.


In [2]:
env = gym.make('CartPole-v0')
print(env.reset())
print(env.observation_space)
print(env.action_space)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[ 0.02197176 -0.01695209  0.00955268 -0.01527069]
Box(4,)
Discrete(2)


In [3]:
model_sample = collections.namedtuple('model_sample',
    ['state', 'action', 'reward'])

In [66]:
class model:
    
    def __init__(self, input_n, output_n, beta=0.0001):
        self.__input_n = input_n
        self.__output_n = output_n
        self.__l_shared = [
            #keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(64, activation='relu')]
        self.__l_policy = [
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(self.__output_n, activation='softmax')]
        self.__l_value = [
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(1, activation='relu')]
        def apply_layers(x, layers):
            last_layer = x
            for l in layers:
                last_layer = l(x)
            return last_layer
        m = m_input = keras.layers.Input((self.__input_n,))
        m = apply_layers(m, self.__l_shared)
        m = apply_layers(m, self.__l_policy)
        self.__m_policy = keras.models.Model([m_input], [m])
        self.__m_policy.compile('adam', 'mse')
        m = m_input = keras.layers.Input((self.__input_n,))
        m = apply_layers(m, self.__l_shared)
        m = apply_layers(m, self.__l_value)
        self.__m_value = keras.models.Model([m_input], [m])
        self.__m_value.compile('adam', 'mse')
        m = m_input = keras.layers.Input((self.__input_n,))
        m_shared = apply_layers(m, self.__l_shared)
        m_policy = apply_layers(m_shared, self.__l_policy)
        m_value = apply_layers(m_shared, self.__l_value)
        self.__m_value_policy = keras.models.Model([m_input], [m_value, m_policy])
        self.__m_value_policy.compile('adam', 'mse')
        m = m_input = keras.layers.Input((self.__input_n,))
        m_value, m_policy = self.__m_value_policy(m)
        m = keras.layers.Concatenate()([m_value, m_policy])
        self.__m_optimizer = keras.optimizers.Adam(clipnorm=5.)
        self.__m_train = keras.models.Model([m_input], [m])
        self.__m_train.compile(self.__m_optimizer,
            lambda y_true, y_pred: model.__loss(y_true, y_pred, beta))
    
    @staticmethod
    def __loss(y_true, y_pred, beta):
        r, action_onehot = y_true[:,:1], y_true[:,1:]
        value, policy = y_pred[:,:1], y_pred[:,1:]
        advantage = r - value
        log_policy = K.log(policy + K.epsilon())
        log_choosen_action_prob = K.sum(action_onehot * log_policy, axis=-1),
        action_loss = -K.mean(log_choosen_action_prob * advantage)
        value_loss = 0.5 * K.mean(K.square(advantage))
        entropy = K.mean(-K.sum(policy * log_policy, axis=-1))
        return action_loss + value_loss - beta * entropy
    
    def train(self, samples, verbose=False):
        self.__m_train.fit(
            x=np.array([s.state for s in samples], dtype=np.float32),
            y=np.hstack([
                np.reshape(np.array([s.reward for s in samples], dtype=np.float32), (-1, 1)),
                keras.utils.to_categorical([s.action for s in samples], num_classes=self.__output_n)]),
            batch_size=32,
            epochs=1,
            verbose=verbose)
    
    def evalute_value(self, state, verbose=False):
        v = self.__m_value.predict(
            np.array([state], dtype=np.float32))[0,0]
        if verbose:
            print(v)
        return v
    
    def get_action_prob(self, state, verbose=False):
        action_prob = self.__m_policy.predict(
            np.array([state], dtype=np.float32))[0]
        if verbose:
            print(action_prob)
        return action_prob

In [71]:
def play(env, m, gamma=0.98, max_steps=1000, verbose=False):
    state_0 = env.reset()
    episode = []
    samples = []
    gamelen = 0
    gamelen_max = 0
    def add_to_samples(episode, done):
        if done:
            discounted_reward = 0.
        else:
            discounted_reward = m.evalute_value(episode[-1].state)
        episode[-1] = model_sample(
                episode[-1].state,
                episode[-1].action,
                discounted_reward)
        for i in reversed(range(len(episode)-1)):
            discounted_reward = episode[i].reward + \
                gamma * discounted_reward
            episode[i] = model_sample(
                episode[i].state,
                episode[i].action,
                discounted_reward)
        samples.extend(episode)
    for i in range(max_steps):
        action_prob = m.get_action_prob(state_0)
        action = int(np.random.choice(
            list(range(action_prob.shape[-1])),
            p=action_prob))
        state_1, reward, done, _ = env.step(action)
        episode.append(model_sample(state_0, action, reward))
        state_0 = state_1
        gamelen += 1
        if done:
            add_to_samples(episode, True)
            episode = []
            state_0 = env.reset()
            gamelen_max = max(gamelen_max, gamelen)
            gamelen = 0
    if episode:
        add_to_samples(episode, False)
        gamelen_max = max(gamelen_max, gamelen)
        gamelen = 0
    if verbose:
        print('max game len', gamelen_max)
    return samples

In [72]:
K.clear_session()
m = model(4, 2, beta=0.0001)

In [73]:
def test(m):
    env = gym.make('CartPole-v0')
    state = env.reset()
    rewards = 0.
    while True:
        action_prob = m.get_action_prob(state)
        action = np.argmax(action_prob)
        state, reward, done, _ = env.step(action)
        rewards += reward
        env.render()
        if done:
            break
        time.sleep(1/30)
    env.close()
    print(rewards)
test(m)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
32.0


In [74]:
for i in range(200):
    samples = play(env, m, max_steps=1000, verbose=True)
    m.train(samples, verbose=True)
    print('epoch {} completed'.format(i))
    #test(m)

max game len 59
Epoch 1/1
1000/1000 [==============================] - 0s 251us/step - loss: 119.7306
epoch 0 completed
max game len 77
Epoch 1/1
1000/1000 [==============================] - 0s 132us/step - loss: 104.4395
epoch 1 completed
max game len 73
Epoch 1/1
1000/1000 [==============================] - 0s 116us/step - loss: 118.9443
epoch 2 completed
max game len 44
Epoch 1/1
1000/1000 [==============================] - 0s 133us/step - loss: 93.3450
epoch 3 completed
max game len 92
Epoch 1/1
1000/1000 [==============================] - 0s 119us/step - loss: 117.4378
epoch 4 completed
max game len 56
Epoch 1/1
1000/1000 [==============================] - 0s 122us/step - loss: 82.9694
epoch 5 completed
max game len 79
Epoch 1/1
1000/1000 [==============================] - 0s 113us/step - loss: 84.2041
epoch 6 completed
max game len 104
Epoch 1/1
1000/1000 [==============================] - 0s 189us/step - loss: 143.5864
epoch 7 completed
max game len 84
Epoch 1/1
1000/1000 [=====

KeyboardInterrupt: 